In [37]:
from tools import load_config
# Import necessary libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


# Load the config.yaml
config = load_config("config.yaml")

# Extract the processed data location
processed_data_location = config["processed_data_location"]

# Load the processed data into a DataFrame
import pandas as pd
df = pd.read_csv(processed_data_location)

# Display the DataFrame
df.head()


,subjid,sex,vomitfl,temp,virusfl,hosp_days,age_years,antibioticfl,probioticfl,analgesicfl,treatment_combination,combination_description,dehsev_Mild dehydration,dehsev_No dehydration,dehsev_Severe dehydration,BMI,vomit_avgfrq
0,1,2,1,37.8,1,3,0.393607,0,0,0,0,No treatments,False,False,True,13.611111,2.5
1,2,1,1,39.0,1,3,1.893607,0,0,0,0,No treatments,True,False,False,13.442554,3.5
2,3,1,0,37.1,1,5,1.868950,0,0,0,0,No treatments,True,False,False,19.333730,NaN
3,4,2,1,37.7,1,2,0.657306,0,0,1,1,Analgesic only,False,False,True,16.568047,2.5
4,7,1,1,36.5,1,1,0.885388,0,0,0,0,No treatments,False,False,True,11.108907,1.5
